In [1]:
import numpy as np
import cv2 as cv
from lxml import etree
import os
from matplotlib import pyplot as plt
import pytesseract

from kiebids.modules.evaluation import load_image_from_url
from kiebids import config

folder_path =  os.path.join(
    config.shared_folder,
    "data/hymdata_sample/20230511T160908__coll.mfn-berlin.de_u_78a081",
)

output_path = "../data/tmp"


In [11]:
max_images = 20
index = 10
files = [f for f in os.listdir(folder_path) if f.endswith(".xml")][index:max_images+index]
def get_image(index):
    if index > len(files):
        raise ValueError("Index out of range")

    filename = files[index]

    file_path = os.path.join(folder_path, filename)
    tree = etree.parse(file_path)
    root = tree.getroot()
    ns = {"ns": root.nsmap[None]} if None in root.nsmap else {}

    comments = root.find(
        ".//ns:Metadata/ns:Comments" if ns else ".//Metadata/Comments",
        namespaces=ns,
    )
    # excluding some fields without assignment
    comments = dict(item.split("=", 1) for item in comments.text.split(", ") if len(item.split("=", 1)) == 2)

    # loading from url
    image_url = comments.get("imgUrl")
    image = None
    if image_url:
        image = load_image_from_url(image_url)
        grayscale_image = image.convert('L')

        # Convert the grayscale PIL image to a NumPy array (of type uint8)
        grayscale_np = np.array(grayscale_image, dtype=np.uint8)
        # image = grayscale_image
    return image, grayscale_np



In [ ]:
image, grayscale_np = get_image(10)

#### resizing

In [ ]:
# resizing for experinments with kernel size
up_points = (grayscale_np.shape[1] * 4, grayscale_np.shape[0] * 4)
resized_image = cv2.resize(grayscale_np, up_points, interpolation= cv2.INTER_LINEAR)

image = resized_image
print(f"original image shape: {grayscale_np.shape}")
print(f"resized image shape: {resized_image.shape}")

## Thresholding and Morph operations
https://docs.opencv2.org/3.4/d7/d4d/tutorial_py_thresholding.html

### original

In [ ]:
plt.imshow(image, cmap='gray')
plt.show()

### Adaptive Thresh

In [ ]:
for i, f in enumerate(files):
    image, grayscale_np = get_image(i)

    denoised = cv2.fastNlMeansDenoising(grayscale_np,None,10,10,7)

    t = 100
    alg = cv2.ADAPTIVE_THRESH_MEAN_C
    alg = cv2.ADAPTIVE_THRESH_GAUSSIAN_C
    t_type = "adaptive"
    if t_type == "adaptive":
        print("adaptive thresholding")
        thresholded_image = cv2.adaptiveThreshold(denoised, 255, alg, 
                                                cv2.THRESH_BINARY, 19, 20)
    else:
        t = 30
        print("binary thresholding")
        ret, thresholded_image = cv2.threshold(denoised, t,255,cv2.THRESH_BINARY)

    kernel = np.ones((3,3),np.uint8)
    opening = cv2.morphologyEx(thresholded_image, cv2.MORPH_OPEN, kernel)
    # removing outer border
    offset = 20
    opening = opening[offset:opening.shape[0]-offset, offset:opening.shape[1]-offset]

    # kernel = np.ones((3,3),np.uint8)
    closing = cv2.morphologyEx(thresholded_image, cv2.MORPH_CLOSE, kernel)

    kernel = np.ones((3,3),np.uint8)
    eroded = cv2.erode(opening, kernel, iterations=1)

    fig, axes = plt.subplots(3, 2, figsize=(10, 10))

    # Plot each image in the respective subplot
    axes[0, 0].imshow(grayscale_np, cmap='gray')
    axes[0, 0].set_title("greyscale image")
    axes[0, 0].axis('off')  # Remove axes for better visualization

    axes[1, 0].imshow(thresholded_image, cmap='gray')
    axes[1, 0].set_title("thresholded image")
    axes[1, 0].axis('off')

    axes[0, 1].imshow(denoised, cmap='gray')
    axes[0, 1].set_title("denoised")
    axes[0, 1].axis('off')

    axes[1, 1].imshow(opening, cmap='gray')
    axes[1, 1].set_title("opening")
    axes[1, 1].axis('off')

    axes[2, 0].imshow(closing, cmap='gray')
    axes[2, 0].set_title("closing")
    axes[2, 0].axis('off')

    axes[2, 1].imshow(eroded, cmap='gray')
    axes[2, 1].set_title("eroded opening")
    axes[2, 1].axis('off')

    # Adjust layout to avoid overlapping
    plt.tight_layout()
    plt.show()
    plt.savefig(f"{output_path}/{f.replace('.xml', '_combined.jpg')}", dpi=300, bbox_inches='tight')            


In [ ]:
opening = cv2.imread("../data/tmp/88dd7970-fbac-4321-a15b-1ad6adfbb269_label_front_0001_label_opening.jpg", cv2.IMREAD_GRAYSCALE)

rgb_image = cv2.cvtColor(opening, cv2.COLOR_GRAY2BGR)
boxes = pytesseract.image_to_boxes(opening)

# contours, _ = cv2.findContours(opening, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours, _ = cv2.findContours(opening, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

print(len(contours))

# cv2.drawContours(rgb_image, contours, -1, (0, 255, 0), 2)

# cv2.imwrite(f"{output_path}/{f.replace('.xml', '_contours.jpg')}", rgb_image)


In [ ]:
opening = cv2.imread("../data/tmp/88dd7970-fbac-4321-a15b-1ad6adfbb269_label_front_0001_label_opening.jpg", cv2.IMREAD_GRAYSCALE)

rgb_image = cv2.cvtColor(opening, cv2.COLOR_GRAY2BGR)

layerNames = [
	"feature_fusion/Conv_7/Sigmoid",
	"feature_fusion/concat_3"]
net = cv2.dnn.readNet()

blob = cv2.dnn.blobFromImage(image, 1.0, (W, H),
	(123.68, 116.78, 103.94), swapRB=True, crop=False)
net.setInput(blob)
(scores, geometry) = net.forward(layerNames)

# (rects, confidences) = decode_predictions(scores, geometry)
# boxes = non_max_suppression(np.array(rects), probs=confidences)


### Opening

In [ ]:
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(thresholded_image, cv2.MORPH_OPEN, kernel)
plt.imshow(opening, cmap='gray')
plt.show()

### Closing

In [ ]:
kernel = np.ones((3,3),np.uint8)
closing = cv2.morphologyEx(thresholded_image, cv2.MORPH_CLOSE, kernel)
plt.imshow(closing, cmap='gray')
plt.show()